In [23]:
import numpy as np
import tensorflow as tf
import tools
import metrics
import keras_losses as kl
import keras_models as kms
import bnn_predictions as bnn

In [15]:
run_ns = range(1,11)
#architecture parameters
num_inputs = 14
num_outputs = 2
layer_sizes = []
m = 64 #total number of records
training_size = 48. / 64.
test_size = 1 - training_size
test_num = int(np.ceil(m * test_size))

In [7]:
#following loops could easily be included into same loop, but currently have them separate for clarity

In [10]:
MPE_test_losses = []
MPE_test_accuracies = []
for run_n in run_ns:
    #load data
    x_test = np.genfromtxt("../../data/kaggle/FIFA_2018_Statistics_x_te_" + str(run_n) + ".csv", delimiter = ',')
    y_test = np.genfromtxt("../../data/kaggle/FIFA_2018_Statistics_y_te_" + str(run_n) + ".csv", delimiter = ',')
    #load keras MPE model
    model_r = tf.keras.models.load_model("../../MPE_examples/saved_keras_models/kaggle/FIFA_2018_Statistics_mlp_1_sm_" + str(run_n) + ".h5", custom_objects={'sum_of_categorical_crossentropy': kl.sum_of_categorical_crossentropy})
    #calculate evaluation metrics for keras MPE
    y_pred_r = model_r.predict(x_test)
    y_test, y_pred_r = tools.check_dtypes(y_test, y_pred_r)
    test_loss_r = metrics.categorical_crossentropy(y_test, y_pred_r) #averaged over test set size
    test_acc_r = metrics.categorical_accuracy(y_test, y_pred_r)
    MPE_test_losses.append(test_loss_r)
    MPE_test_accuracies.append(test_acc_r)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [16]:
#this is for testing purposes 
#model.evaluate should be used with caution, as if mode contains regularisation, the method will include it
score_r = model_r.evaluate(x_test, y_test, verbose=0, batch_size=test_num)
print('Test loss:', score_r[0])
print('Test accuracy:', score_r[1])

('Test loss:', 10.246844291687012)
('Test accuracy:', 0.875)


In [21]:
print tools.get_km_weight_magnitudes(model_r)

NameError: name 'weight_arr_r' is not defined

In [20]:
#cpp bnn

In [27]:
# bnn expected estimates
bnn_test_losses = []
bnn_test_accuracies = []
for seed_n in seed_ns:
    chains_file = '../cpp_models/cpp_chains/FIFA_2018_Statistics_slp_sm_' + str(seed_n) + '.txt'
    x_test = np.genfromtxt("../../data/kaggle/FIFA_2018_Statistics_x_te_" + str(seed_n) + ".csv", delimiter=',')
    y_test = np.genfromtxt("../../data/kaggle/FIFA_2018_Statistics_y_te_" + str(seed_n) + ".csv", delimiter=',')
    bmodel = kms.slp_sm(num_inputs, num_outputs, layer_sizes)
    bnn_model = bnn.bnn_predictor(bmodel, x_test, y_test, chains_file)
    by_pred = bnn_model.predictions_expectation()
    y_test, by_pred = tools.check_dtypes(y_test, by_pred)
    b_test_loss = metrics.categorical_crossentropy(y_test, by_pred)
    b_test_acc = metrics.categorical_accuracy(y_test, by_pred)
    bnn_test_losses.append(b_test_loss)
    bnn_test_accuracies.append(b_test_acc)

In [29]:
# bnn MPE estimates
bnn_MPE_test_losses = []
bnn_MPE_test_accuracies = []
for seed_n in seed_ns:
    chains_file = '../cpp_models/cpp_chains/FIFA_2018_Statistics_slp_sm_' + str(seed_n) + '.txt'
    x_test = np.genfromtxt("../../data/kaggle/FIFA_2018_Statistics_x_te_" + str(seed_n) + ".csv", delimiter=',')
    y_test = np.genfromtxt("../../data/kaggle/FIFA_2018_Statistics_y_te_" + str(seed_n) + ".csv", delimiter=',')
    bmodel = kms.slp_sm(num_inputs, num_outputs, layer_sizes)
    bnn_model = bnn.bnn_predictor(bmodel, x_test, y_test, chains_file)
    by_pred = bnn_model.MPE_prediction()
    y_test, by_pred = tools.check_dtypes(y_test, by_pred)
    b_test_loss = metrics.categorical_crossentropy(y_test, by_pred)
    b_test_acc = metrics.categorical_accuracy(y_test, by_pred)
    bnn_MPE_test_losses.append(b_test_loss)
    bnn_MPE_test_accuracies.append(b_test_acc)

In [99]:
print tools.get_km_weight_magnitudes(bnn_model.model)

[[-1.8274463   1.0707233 ]
 [ 1.7094133   2.1497848 ]
 [ 0.16490114  2.1739173 ]
 [-1.3065757  -0.5605142 ]
 [ 0.39965272  1.2336111 ]
 [ 0.20807227  0.36302057]
 [-1.7788959  -1.1387717 ]
 [ 0.49126852  2.0472205 ]
 [-0.01662266  0.9817281 ]
 [-1.993626   -0.17394267]
 [ 1.3757656   1.7654557 ]
 [ 0.14453389 -1.1336504 ]
 [-0.509835   -0.45083138]
 [ 0.32196507  2.0590575 ]]
6.756143
[0.76898944 1.6223899 ]


In [13]:
print MPE_test_losses
print np.mean(MPE_test_accuracies)

[0.32759630838795345, 0.4225802663773003, 0.12131025542980348, 0.30824884459329777, 0.11625859189461707, 0.23641955767032724, 0.16409620996177599, 0.22252846215936345, 0.2170064017972021, 0.2708099053281938]
0.88125


In [30]:
print bnn_test_losses
print bnn_test_accuracies

[0.366062825470115, 0.6284091941729878, 0.24924489948015272, 0.5710904364299889, 0.24501502361292296, 0.3579497096714941, 0.27207251425240053, 0.3570041982039275, 0.34970751339715545, 0.3565284383714001]
[0.75, 0.625, 0.9375, 0.625, 0.9375, 0.8125, 0.875, 0.8125, 0.8125, 0.8125]


In [31]:
print bnn_MPE_test_losses
print bnn_MPE_test_accuracies

[0.8409075339178537, 0.8671017332912981, 0.550785078166951, 0.7411403889394357, 0.25798395642442773, 0.7648644009996977, 0.27389507183596595, 0.6367651840596237, 0.9569289182702858, 0.5288760076862427]
[0.5625, 0.625, 0.75, 0.625, 0.9375, 0.8125, 0.875, 0.8125, 0.625, 0.75]
